<a href="https://colab.research.google.com/github/NityaVerma19/Customer-Purchase-Decision-/blob/master/Consumer_behaviour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split


#Generating a dataset

### Product Info

In [171]:
#green packaging

def generate_random_series(length, seed=42):
    random.seed(seed)
    return [random.choice([0, 1]) for _ in range(length)]

def add_binary_noise(series, noise_prob=0.05, seed=None):
    if seed is not None:
        random.seed(seed)
    noisy_series = []
    for value in series:
        if random.random() < noise_prob:
            noisy_series.append(1 - value)  # Flip the value with probability noise_prob
        else:
            noisy_series.append(value)
    return noisy_series

length = 3000
green_pack = generate_random_series(length, seed=43)

noisy_green_pack = add_binary_noise(green_pack, noise_prob=1, seed=43)

In [172]:
#purchase decision - yes/no

def generate_random_series(length, seed = 42):
    random.seed(seed
    )
    return [random.choice([0, 1]) for _ in range(length)]

def add_noise(series, noise_prob=0.05, seed=None):
    if seed is not None:
        random.seed(seed)
    noisy_series = []
    for value in series:
        if random.random() < noise_prob:
            noisy_series.append(1 - value)  # Flip the value with probability noise_prob
        else:
            noisy_series.append(value)
    return noisy_series

length = 3000
purchase = generate_random_series(length, seed=43)

noisy_purchase = add_binary_noise(green_pack, noise_prob=1, seed=43)


In [173]:
#type of product - 'Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing'


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['Beverage', 'Food' , 'Beauty', 'Personal hygiene', 'Paper products' , 'Clothing']
series_length = 3000

product = generate_random_series(word_list, series_length, seed = 42)


In [174]:
import numpy as np
import pandas as pd
import random

def generate_random_series(lower_bound, upper_bound, count, seed=42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)

#for adding noise to the dataset so that it becomes more authentic
def add_multiplicative_noise(series, noise_scale=0.05, seed=None):
    if seed is not None:
        random.seed(seed)
    noise_factors = np.random.normal(loc=1.0, scale=noise_scale, size=len(series))
    return series * noise_factors

lower_bound = 500
upper_bound = 5000
count = 3000

price = generate_random_series(lower_bound, upper_bound, count, seed=42)

noisy_price = add_multiplicative_noise(price, noise_scale=0.05, seed=42)

# Ensure that the resulting prices remain positive
noisy_price = noisy_price.clip(lower=lower_bound)  # Clip to ensure prices don't go below lower_bound

### Customer Info

In [175]:
#Age

import numpy as np
import pandas as pd
import random

def generate_random_series(lower_bound, upper_bound, count, seed=42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)

def add_discrete_noise(series, noise_scale=2, seed=None):
    if seed is not None:
        random.seed(seed)
    noise = np.random.randint(-noise_scale, noise_scale + 1, len(series))
    noisy_series = series + noise
    noisy_series = np.clip(noisy_series, lower_bound, upper_bound)
    return pd.Series(noisy_series)

lower_bound = 18
upper_bound = 50
count = 3000


age = generate_random_series(lower_bound, upper_bound, count, seed=42)

noisy_age = add_discrete_noise(age, noise_scale=2, seed=42)

In [176]:
#Gender


def generate_random_series(words, length, seed = 42):
    random.seed(seed)
    series = [random.choice(words) for _ in range(length)]
    return series

word_list = ['M', 'F']
series_length = 3000

gender = generate_random_series(word_list, series_length, seed = 42)


In [177]:
#income = 15k to 150k

def generate_random_series(lower_bound, upper_bound, count, seed=42):
    random.seed(seed)
    random_numbers = np.random.randint(lower_bound, upper_bound + 1, count)
    return pd.Series(random_numbers)

def add_multiplicative_noise(series, noise_scale=0.05, seed=None):
    if seed is not None:
        random.seed(seed)
    noise_factors = np.random.normal(loc=1.0, scale=noise_scale, size=len(series))
    return series * noise_factors

lower_bound = 15000
upper_bound = 150000
count = 3000

income = generate_random_series(lower_bound, upper_bound, count, seed=42)
noisy_income = add_multiplicative_noise(income, noise_scale=0.05, seed=42)

noisy_income = np.round(noisy_income / 10) * 10

In [178]:
#customer id
numbers_sequence = range(1, 3000)
cust_id = pd.Series(numbers_sequence)


*Merging the series*

In [179]:
df = pd.DataFrame({'cust_id': cust_id,'product': product, 'price': noisy_price, 'green_pack': noisy_green_pack, 'age': noisy_age,'gender': gender, 'income': noisy_income, 'purchase': noisy_purchase })
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,2749.059548,1,18,M,126410.0,1
1,2.0,Beverage,2686.853410,0,23,M,122560.0,0
2,3.0,Beverage,2397.096678,1,24,F,63640.0,1
3,4.0,Clothing,3802.393746,0,36,M,59160.0,0
4,5.0,Beauty,3595.559992,0,47,M,90720.0,0


In [180]:
# Downloading the dataframe

#df.to_csv('df.csv', index=False)

In [181]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,2749.059548,1,18,M,126410.0,1
1,2.0,Beverage,2686.853410,0,23,M,122560.0,0
2,3.0,Beverage,2397.096678,1,24,F,63640.0,1
3,4.0,Clothing,3802.393746,0,36,M,59160.0,0
4,5.0,Beauty,3595.559992,0,47,M,90720.0,0


In [182]:
df.shape

(3000, 8)


---





# Logistic Regression


In [183]:
df.head()

,cust_id,product,price,green_pack,age,gender,income,purchase
0,1.0,Clothing,2749.059548,1,18,M,126410.0,1
1,2.0,Beverage,2686.853410,0,23,M,122560.0,0
2,3.0,Beverage,2397.096678,1,24,F,63640.0,1
3,4.0,Clothing,3802.393746,0,36,M,59160.0,0
4,5.0,Beauty,3595.559992,0,47,M,90720.0,0


In [204]:
#feature variable

X = df.drop(columns = ['cust_id', 'purchase'], axis= 1)


#target variable

y = df.purchase.values



In [205]:
#splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [206]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

((2400, 6), (600, 6))

*Feature engineering*

In [207]:
# display categorical variables

categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

['product', 'gender']

In [208]:
#display numerical variables

numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

['price', 'green_pack', 'age', 'income']

In [209]:
X_train.describe()

,price,green_pack,age,income
count,2400.000000,2400.000000,2400.000000,2400.000000
mean,2717.553655,0.492083,33.748333,82023.845833
std,1305.317598,0.500042,9.641165,38764.642138
min,500.000000,0.000000,18.000000,14200.000000
25%,1584.527053,0.000000,25.000000,48610.000000
50%,2700.851902,0.000000,34.000000,82765.000000
75%,3852.506957,1.000000,42.000000,114590.000000
max,5486.534221,1.000000,50.000000,169680.000000


In [210]:
cols = X_train.columns
cols

Index(['product', 'price', 'green_pack', 'age', 'gender', 'income'], dtype='object')

*Endoding categorical variable*

In [211]:
import category_encoders as ce

encoder = ce.BinaryEncoder(cols=['product', 'gender'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

In [212]:
X_train.head()

,product_0,product_1,product_2,price,green_pack,age,gender_0,gender_1,income
2370,0,0,1,584.363153,0,42,0,1,36080.0
1774,0,1,0,1706.197415,1,40,1,0,135980.0
731,0,1,1,1324.566682,0,48,1,0,106180.0
271,1,0,0,2824.194095,1,28,1,0,23910.0
1077,1,0,1,4110.100918,0,31,1,0,139950.0


In [213]:
X_train = pd.concat([X_train[numerical], X_train[['product_0', 'product_1', 'product_2', 'gender_0', 'gender_1']]], axis=1)

X_train.head()


,price,green_pack,age,income,product_0,product_1,product_2,gender_0,gender_1
2370,584.363153,0,42,36080.0,0,0,1,0,1
1774,1706.197415,1,40,135980.0,0,1,0,1,0
731,1324.566682,0,48,106180.0,0,1,1,1,0
271,2824.194095,1,28,23910.0,1,0,0,1,0
1077,4110.100918,0,31,139950.0,1,0,1,1,0


In [214]:
X_test = pd.concat([X_test[numerical], X_test[['product_0', 'product_1', 'product_2', 'gender_0', 'gender_1']]], axis=1)

X_test.head()


,price,green_pack,age,income,product_0,product_1,product_2,gender_0,gender_1
311,4938.676125,1,28,82050.0,0,1,1,1,0
1025,3809.154246,0,39,100960.0,1,0,0,0,1
1587,1486.293311,1,32,152100.0,0,1,0,0,1
2941,3416.250302,1,43,60940.0,0,0,1,0,1
2980,3782.453174,1,30,81540.0,1,0,0,1,0


*Feature Scaling*

In [215]:
cols = X_train.columns

In [216]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

In [217]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [218]:
X_test = pd.DataFrame(X_test, columns=[cols])

*Model Training*

In [219]:
# train a logistic regression model on the training set
from sklearn.linear_model import LogisticRegression


# instantiate the model
logreg = LogisticRegression(penalty = 'l1', C = 1000, solver='liblinear', random_state=1)


# fit the model
logreg.fit(X_train, y_train)

LogisticRegression(C=1000, penalty='l1', random_state=1, solver='liblinear')

*Predict Results*

In [220]:
y_pred_test = logreg.predict(X_test)

y_pred_test

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,

In [221]:
# probability of getting output as 0 - no purchase

logreg.predict_proba(X_test)[:,0]

array([1.19574201e-05, 9.99992143e-01, 4.25947142e-06, 2.62938086e-06,
       3.09040190e-06, 8.10576232e-06, 9.99995395e-01, 9.99985274e-01,
       9.99991660e-01, 1.08856603e-05, 1.30997481e-05, 2.38766650e-05,
       9.99997884e-01, 5.95707247e-06, 3.03502996e-06, 9.99975154e-01,
       9.99997286e-01, 2.55399670e-06, 2.63860735e-06, 9.99988693e-01,
       9.99996616e-01, 9.99994526e-01, 9.99998945e-01, 9.99988796e-01,
       4.61711966e-06, 5.31664868e-06, 9.99976219e-01, 9.99977514e-01,
       1.69362497e-06, 2.96380522e-06, 1.02689983e-05, 9.99992684e-01,
       3.62824799e-06, 9.99991700e-01, 9.99979861e-01, 9.99994440e-01,
       1.19659553e-05, 9.99971626e-01, 9.99989705e-01, 9.99979911e-01,
       1.25844282e-05, 7.54090586e-06, 5.88916685e-06, 2.24545543e-06,
       9.99991992e-01, 1.88246100e-06, 1.07034998e-05, 9.99972109e-01,
       9.99985254e-01, 4.14042713e-06, 9.99997486e-01, 2.64841468e-06,
       9.99995322e-01, 9.99990745e-01, 3.24804289e-06, 9.99994078e-01,
      

In [222]:
# probability of getting output as 1 - purchase

logreg.predict_proba(X_test)[:,1]

array([9.99988043e-01, 7.85686544e-06, 9.99995741e-01, 9.99997371e-01,
       9.99996910e-01, 9.99991894e-01, 4.60451532e-06, 1.47256131e-05,
       8.34027111e-06, 9.99989114e-01, 9.99986900e-01, 9.99976123e-01,
       2.11639420e-06, 9.99994043e-01, 9.99996965e-01, 2.48462381e-05,
       2.71444389e-06, 9.99997446e-01, 9.99997361e-01, 1.13066614e-05,
       3.38426257e-06, 5.47412139e-06, 1.05481879e-06, 1.12043489e-05,
       9.99995383e-01, 9.99994683e-01, 2.37808549e-05, 2.24860109e-05,
       9.99998306e-01, 9.99997036e-01, 9.99989731e-01, 7.31559047e-06,
       9.99996372e-01, 8.30005885e-06, 2.01391276e-05, 5.56012027e-06,
       9.99988034e-01, 2.83743572e-05, 1.02949657e-05, 2.00890828e-05,
       9.99987416e-01, 9.99992459e-01, 9.99994111e-01, 9.99997755e-01,
       8.00751252e-06, 9.99998118e-01, 9.99989297e-01, 2.78913692e-05,
       1.47460658e-05, 9.99995860e-01, 2.51353844e-06, 9.99997352e-01,
       4.67826841e-06, 9.25473945e-06, 9.99996752e-01, 5.92197982e-06,
      

### Checking Accuracy

In [223]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model accuracy score: 1.0000
